# Домашнее задание к лекции «Основы парсинга и работы с API»

## Задание 1.
### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [4]:
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'
params = {
    'q': 'python'
}
# получение результатов запроса для 'python'
req = requests.get(URL, params=params)
req.text


'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Результаты поиска по запросу «python» / Хабр</title>\n\n\n\n\n\n<meta name=\'yandex-verification\' content=\'71593b225aeafc4e\' />\n<meta name=\'referrer\' content=\'unsafe-url\' />\n<meta name="pocket-site-verification" content="ed24b2b9721edf0a282c5b4a3232c4" />\n<meta name="biu" content="https://dr.habracdn.net/habr/603df53d/images/">\n\n<style type="text/css">\n  @font-face{font-family:\'Fira Sans\';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr.habracd

In [5]:
# парсинг полученного текста
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Результаты поиска по запросу «python» / Хабр</title>
<meta content="71593b225aeafc4e" name="yandex-verification">
<meta content="unsafe-url" name="referrer"/>
<meta content="ed24b2b9721edf0a282c5b4a3232c4" name="pocket-site-verification"/>
<meta content="https://dr.habracdn.net/habr/603df53d/images/" name="biu"/>
<style type="text/css">
  @font-face{font-family:'Fira Sans';font-style:normal;font-weight:500;src:url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-medium.eot);src:local("Fira Sans Medium"),local("FiraSans-Medium"),url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-medium.eot?#iefix) format("embedded-opentype"),url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-medium.woff2) format("woff2"),url(https://dr.habracdn.net/habr/603df53d/fonts/FiraSans/firaSans-me

In [6]:
# текст с блоками постов
post_bloks = soup.find_all('article', class_='post')
post_bloks

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/denis-19/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/6a0/f31/235/6a0f31235bb26134f3c7a741c03e4efb.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">denis-19</span>
 </a>
 <span class="post__time">21 апреля 2020 в 18:35</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/news/t/498364/">Вышел <em class="searched-item">Python</em> 2.7.18, последний релиз ветки <em class="searched-item">Python</em> 2.x</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/python/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Python

In [7]:
# текст ссылок на посты
post_links_text = list(map(lambda x: x.find('a', class_='post__title_link'), post_bloks))
post_links_text

[<a class="post__title_link" href="https://habr.com/ru/news/t/498364/">Вышел <em class="searched-item">Python</em> 2.7.18, последний релиз ветки <em class="searched-item">Python</em> 2.x</a>,
 <a class="post__title_link" href="https://habr.com/ru/news/t/531402/"><em class="searched-item">Python</em> как компилируемый статически типизированный язык программирования</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/itsumma/news/t/490834/">...  начале этого года <em class="searched-item">Python</em> сместил Java и ...</a>,
 <a class="post__title_link" href="https://habr.com/ru/news/t/505096/">Вышла версия 0.0.2 snakeware — дистрибутива Linux, в котором всё работает через <em class="searched-item">Python</em></a>,
 <a class="post__title_link" href="https://habr.com/ru/news/t/527858/">Создатель <em class="searched-item">Python</em> Гвидо ван Россум выходит на работу в Microsoft</a>,
 <a class="post__title_link" href="https://habr.com/ru/news/t/473926/">Создатель <em class=

In [8]:
# список ссылок
all_refs = list(map(lambda x: x.get('href'), post_links_text))
all_refs

['https://habr.com/ru/news/t/498364/',
 'https://habr.com/ru/news/t/531402/',
 'https://habr.com/ru/company/itsumma/news/t/490834/',
 'https://habr.com/ru/news/t/505096/',
 'https://habr.com/ru/news/t/527858/',
 'https://habr.com/ru/news/t/473926/',
 'https://habr.com/ru/news/t/519414/',
 'https://habr.com/ru/company/skillfactory/blog/544430/',
 'https://habr.com/ru/post/147281/',
 'https://habr.com/ru/post/168083/',
 'https://habr.com/ru/post/205944/',
 'https://habr.com/ru/company/mailru/blog/337364/',
 'https://habr.com/ru/company/ods/blog/348260/',
 'https://habr.com/ru/company/otus/blog/425233/',
 'https://habr.com/ru/company/mailru/blog/454324/',
 'https://habr.com/ru/company/intersystems/blog/486984/',
 'https://habr.com/ru/company/mailru/blog/538706/',
 'https://habr.com/ru/post/46350/',
 'https://habr.com/ru/post/161931/',
 'https://habr.com/ru/post/167261/']

In [9]:
# оборачиваем в функцию получения списка ссылок
KEYWORDS = ['python', 'парсинг']
URL = 'https://habr.com/ru/search/'

def get_links(URL, KEYWORDS):
    '''Возвращает список ссылок на посты из URL содержащих любое слово из KEYWORDS'''
    all_refs = []
    params = {}
    for word in KEYWORDS:
        params['q'] = word
        req = requests.get(URL, params=params)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        post_bloks = soup.find_all('article', class_='post')
        all_refs += list(map(lambda x: x.find('a', class_='post__title_link').get('href'), post_bloks))
    return all_refs

all_links = get_links(URL, KEYWORDS)
all_links

['https://habr.com/ru/news/t/498364/',
 'https://habr.com/ru/news/t/531402/',
 'https://habr.com/ru/company/itsumma/news/t/490834/',
 'https://habr.com/ru/news/t/505096/',
 'https://habr.com/ru/news/t/527858/',
 'https://habr.com/ru/news/t/473926/',
 'https://habr.com/ru/news/t/519414/',
 'https://habr.com/ru/company/skillfactory/blog/544430/',
 'https://habr.com/ru/post/147281/',
 'https://habr.com/ru/post/168083/',
 'https://habr.com/ru/post/205944/',
 'https://habr.com/ru/company/mailru/blog/337364/',
 'https://habr.com/ru/company/ods/blog/348260/',
 'https://habr.com/ru/company/otus/blog/425233/',
 'https://habr.com/ru/company/mailru/blog/454324/',
 'https://habr.com/ru/company/intersystems/blog/486984/',
 'https://habr.com/ru/company/mailru/blog/538706/',
 'https://habr.com/ru/post/46350/',
 'https://habr.com/ru/post/161931/',
 'https://habr.com/ru/post/167261/',
 'https://habr.com/ru/post/446488/',
 'https://habr.com/ru/post/521646/',
 'https://habr.com/ru/post/340302/',
 'https:

In [10]:
# даты
# заголовоки

def get_articles(all_links):
    '''По найденным ссылкам из поискового запроса возвращает Dataframe
    со столбцами'date', 'title', 'link' для найденных результатов'''
    df_articles = pd.DataFrame()
    for link in all_links:
        req = requests.get(link)
        time.sleep(0.3)
        soup = BeautifulSoup(req.text, 'html.parser')
        date = pd.to_datetime(soup.find('span', class_='post__time').get('data-time_published'))
        title = soup.find('span', class_='post__title-text').text
        row = {'date': date, 'title': title, 'link': link}
        df_articles = pd.concat([df_articles, pd.DataFrame([row])])
    return df_articles.reset_index(drop=True)
df_articles = get_articles(all_links)
df_articles

ConnectionError: HTTPSConnectionPool(host='habr.com', port=443): Max retries exceeded with url: /ru/company/mailru/blog/454324/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001C8C5A70220>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))

# Задание 2.¶
## Обязательная часть¶

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода: EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

## Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте. Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>



In [ ]:
# EMAIL = {'emailAddresses': ['xxx@x.ru', 'yyy@y.com']}
# URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
# headers = {"Vaar-Version": "0",
#           "Vaar-Header-App-Product-Name": "hackcheck-web-avast",
#           "Vaar-Header-App-Product": "hackcheck-web-avast",
#           "Vaar-header-App-Build-Version": "1.0.0"}
# req = requests.post(URL, json=EMAIL, headers=headers)
# req.text

In [ ]:
# req.json()

In [ ]:
# df = pd.DataFrame(req.json()['data'])
# df.info()

In [ ]:
# df


In [15]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']

url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-awast', 'Vaar-Header-App-Product-Name': 'hackcheck-web-avast', 'Vaar-Header-App-Build-Version': '1.0.0'}
addresses = {'emailAddresses': EMAIL}
res = requests.post(url, json=addresses, headers=headers)
df = pd.DataFrame(res.json()['breaches']).T.reset_index()
df = df[['site', 'description', 'publishDate']]
df['email'] = [', '.join(el.keys()) for el in list(res.json()['data'].values())]


In [17]:
df

,site,description,publishDate,email
0,azcentral.com,"At an unconfirmed date, online Arizona newspap...",2020-01-03T00:00:00Z,xxx@x.ru
1,wishbone.io,"In January 2020, the online poll website Wishb...",2020-05-28T00:00:00Z,"xxx@x.ru, yyy@y.com"
2,myheritage.com,"In October 2017, a customer database belonging...",2017-11-04T00:00:00Z,xxx@x.ru
3,forums.vkmonline.com,"At an unconfirmed date, the Russian-language m...",2021-02-11T00:00:00Z,yyy@y.com
4,canva.com,"In May 2019, graphic-design site Canva's datab...",2019-06-13T00:00:00Z,yyy@y.com
5,parapa.mail.ru,"In July and August 2016, two criminals execute...",2017-02-14T00:00:00Z,yyy@y.com
6,dropbox.com,Cloud storage company Dropbox suffered a major...,2016-10-24T00:00:00Z,yyy@y.com
7,linkedin.com,"In 2012, online professional networking platfo...",2016-10-21T00:00:00Z,yyy@y.com
8,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra...",2017-03-01T00:00:00Z,yyy@y.com
9,zynga.com,"In September 2019, the game developer Zynga wa...",2019-10-17T00:00:00Z,yyy@y.com
